In [ ]:
from experiments.attn.victory import *

In [ ]:
device = 'cuda'
D = 32
B = 8*1024
T = 5000

worlds = Hex.initial(B, boardsize=7)

boardsize = worlds.boardsize
model = common.AttnModel(common.PosActions, boardsize, D).to(device)

opt = torch.optim.Adam(model.parameters(), lr=1e-2)

with tqdm(total=T) as pbar:
    for t, b in zip(range(T), batchgen(worlds, B)):
        targets = terminal_actions(b.worlds)
        outputs = model(b.worlds.obs)

        infs = torch.full_like(targets, np.inf)
        loss = -outputs.reshape(B, -1).where(targets == 1., infs).min(-1).values.mean()

        opt.zero_grad()
        loss.backward()
        opt.step()

        pbar.update(1)
        pbar.set_description(f'{loss:.2f}')



In [ ]:
targets = terminal_actions(b.worlds)
outputs = model(b.worlds.obs)

loss = -outputs.reshape(B, -1).mul(targets).sum(-1)

bad = loss.argsort().cpu().numpy()[::-1]

In [ ]:
plot(bad[-1], b.worlds.obs, targets, outputs, model.attns)

In [ ]:
loss.detach().cpu().numpy()[bad]